# Desafio Prático Neurotech

## Autor: Danilo Vaz

Nesse notebook, foi criado uma API, em python, que recebe 2 requisições, cuja, a finalidade é fazer o monitoramento do desempenho de um modelo.
- A primeira requisação tem no seu _body_ uma lista de registros, os registros são representados por 1 dicionário, cujas chave são os atributos e o endereço para o modelo pré-treinado. Ela retorna o valor ROC do modelo no conjunto de registros e uma lista com quantidade de registros por mês, os meses são representados pelo seu respectivo número: 01 - Janeiro, 02 - Fevereiro... . 
- A segunda requisição tem no seu _body_ o endereço para uma base armazenda localmente e o endereço do modelo pré-treinado. Ela retorna a distância estatística KS entre a distribuição da base passada e a base de teste.

### Problemas enfrentados
- Não foi possivel, rodar o código no notebook, pois ele não estava linkado com o ambiente virtual criado para esse desafio, no entando, o código foi validado no formato .py, seguirá fotos do funcionamento no github
- Na base oot, havia um valor para um dos atributos que não havia sido visto no treinamento, por isso o encoder do modelo pré-treinado não conseguio processar os dados. Tentei algumas soluções, por exemplo: substituir o valor que estava dando erro por outro presente na base treino, remover a coluna que dava erro, porém não uma delas foi frutífera. Uma solução para esse problema seria por a flag _handle_unknown='ignore'_ na declaração do OneHotEncoder().

### Seção de import das bibliotecas e funções que foram usadas

In [ ]:
import uvicorn
import pickle, json
from fastapi import FastAPI
import pandas as pd
import numpy as np
from pydantic import BaseModel
from typing import List, Union
from fastapi import APIRouter
from scipy.stats import ks_2samp
from sklearn.metrics import roc_auc_score
import gzip
import csv

### Decalração da tipagem dos objetos que serão recebido nas requisições

In [ ]:
class Registers(BaseModel):
    VAR2: Union[str, None]
    IDADE: Union[float, None]
    VAR5: Union[str, None]
    VAR6: Union[float, None]
    VAR7: Union[float, None]
    VAR8: Union[str, None]
    VAR9: Union[str, None]
    VAR10: Union[str, None]
    VAR11: Union[float, None]
    VAR12: Union[float, None]
    VAR14: Union[float, None]
    VAR15: Union[float, None]
    VAR16: Union[float, None]
    VAR18: Union[float, None]
    VAR19: Union[float, None]
    VAR22: Union[float, None]
    VAR24: Union[float, None]
    VAR25: Union[float, None]
    VAR32: Union[str, None]
    VAR39: Union[float, None]
    VAR40: Union[float, None]
    VAR41: Union[float, None]
    VAR42: Union[float, None]
    VAR47: Union[float, None]
    VAR49: Union[str, None]
    VAR50: Union[str, None]
    VAR51: Union[str, None]
    VAR52: Union[str, None]
    VAR53: Union[str, None]
    VAR54: Union[str, None]
    VAR55: Union[str, None]
    VAR56: Union[str, None]
    VAR57: Union[str, None]
    VAR58: Union[str, None]
    VAR59: Union[str, None]
    VAR60: Union[str, None]
    VAR61: Union[str, None]
    VAR62: Union[str, None]
    VAR63: Union[str, None]
    VAR64: Union[str, None]
    VAR65: Union[str, None]
    VAR66: Union[str, None]
    VAR67: Union[str, None]
    VAR68: Union[str, None]
    VAR69: Union[str, None]
    VAR70: Union[str, None]
    VAR71: Union[str, None]
    VAR72: Union[str, None]
    VAR73: Union[str, None]
    VAR74: Union[str, None]
    VAR75: Union[str, None]
    VAR76: Union[str, None]
    VAR77: Union[str, None]
    VAR78: Union[str, None]
    VAR79: Union[str, None]
    VAR80: Union[str, None]
    VAR81: Union[str, None]
    VAR82: Union[str, None]
    VAR83: Union[str, None]
    VAR84: Union[str, None]
    VAR85: Union[str, None]
    VAR86: Union[str, None]
    VAR87: Union[str, None]
    VAR88: Union[str, None]
    VAR89: Union[str, None]
    VAR90: Union[str, None]
    VAR91: Union[str, None]
    VAR92: Union[str, None]
    VAR93: Union[str, None]
    VAR94: Union[str, None]
    VAR95: Union[str, None]
    VAR96: Union[str, None]
    VAR97: Union[str, None]
    VAR98: Union[str, None]
    VAR99: Union[str, None]
    VAR100: Union[str, None]
    VAR101: Union[str, None]
    VAR102: Union[str, None]
    VAR103: Union[str, None]
    VAR104: Union[str, None]
    VAR105: Union[str, None]
    VAR106: Union[str, None]
    VAR107: Union[str, None]
    VAR108: Union[str, None]
    VAR109: Union[str, None]
    VAR110: Union[str, None]
    VAR111: Union[str, None]
    VAR112: Union[str, None]
    VAR113: Union[str, None]
    VAR114: Union[str, None]
    VAR115: Union[str, None]
    VAR116: Union[str, None]
    VAR117: Union[str, None]
    VAR118: Union[str, None]
    VAR119: Union[str, None]
    VAR120: Union[str, None]
    VAR121: Union[str, None]
    VAR122: Union[str, None]
    VAR123: Union[str, None]
    VAR124: Union[str, None]
    VAR125: Union[str, None]
    VAR126: Union[str, None]
    VAR127: Union[str, None]
    VAR128: Union[str, None]
    VAR129: Union[str, None]
    VAR130: Union[str, None]
    VAR131: Union[str, None]
    VAR132: Union[str, None]
    VAR133: Union[str, None]
    VAR134: Union[str, None]
    VAR135: Union[str, None]
    VAR136: Union[str, None]
    VAR137: Union[str, None]
    VAR138: Union[str, None]
    VAR139: Union[str, None]
    VAR140: Union[str, None]
    VAR141: Union[float, None]
    VAR142: Union[str, None]
    REF_DATE: Union[str, None]
    TARGET: Union[int, None]

class ListRegisters(BaseModel):
    registers: List[Registers]
    model: str

class EndpointAderencia(BaseModel):
    base: str
    model: str

### Declaração das rotas e criação do servidor na porta 8001 

1ª Rota: Rota performance, ela faz uma chamada para a função performance, passando a lista de registro e o endereço do modelo. Recebe como retorno o ROC score e a lista com a frequência por mês e retorna esse dois valores para quem fez a requisição.
2ª Rota: Rota aderencia, ela faz a chamada para a função aderencia, passando o endereço da base e o endereço do modelo. Retorna o valor KS que representa a distância entre a distribuição das bases

In [ ]:
ader_router = APIRouter(prefix="/aderencia")
perf_router = APIRouter(prefix="/performance")

router_api = APIRouter()

# Routers from each endpoint
router_api.include_router(perf_router)
router_api.include_router(ader_router)

@app.post("/performance")
def postPerformance(req: ListRegisters):
    rocAuc, freq_list = performance(req.registers, req.model)
    return {"Roc Auc": f"{rocAuc}", "Frequencia por mes": f"{freq_list}"}

@app.post("/aderencia")
def postAderencia(req: EndpointAderencia):
    value = aderencia(req.base, req.model)
    return {"Ks distance": f"{value}"}


app.include_router(router_api, prefix="/v1")


uvicorn.run(app, host="0.0.0.0", port=8001)

### Funções auxiliares

- contador_meses: função que era processar a lista de registros e retorna uma lista com a frequência de registros por mês
- pre_processing: função que pre processa uma massa de dados, retirando os valores nan e preenchendo-os por meio de interpolação
- probabilidades: função de calcula a probalidade da classe 1 para cada dos registro de um conjunto de dados 

In [ ]:
def contador_meses(df):
    meses = []
    for i in range(1,13):
        if i>9:
            meses.append((f'{i}: ' + str(len(df[df['REF_DATE'].str.contains(f"-{i}-")]))))
        else:
            meses.append((f'0{i}: ' + str(len(df[df['REF_DATE'].str.contains(f"-0{i}-")]))))
    return meses

def pre_processing(df):
    num_cols = df._get_numeric_data().columns
    cat_cols = set(df.columns) - set(num_cols)
    for i in num_cols:
        if df[i].isna().any() == True:
            df[i] = df[i].interpolate(limit_direction='both')

    for i in cat_cols:
        df[i] = df[i].astype('category')
        df[i] = df[i].cat.codes.replace(-1, np.nan).interpolate(limit_direction='both').astype(int).astype('category') \
                     .cat.rename_categories(df[i].cat.categories)
    return df

def probabilidades(df, model):
    probabilidades = []
    print(df[df['VAR79'] == 'MUITO PROXIMO'])
    probs_array = model.predict_proba(df)
    for i in probs_array:
        probabilidades.append(i[1])
    return probabilidades

### Funções responsáveis por processar e retornar para a API cada requisição 
- performance: função da primeira rota, recebe um conjunto de registros, os transforma em dataframe para calcular o ROC score e também retorna a lista de frequência por mês.
- aderencia: função da segunda rota, recebe o endereço de um base de dados e do modelo pré-treinado, faz o pré-processamento tanto da base passada tanto da base de teste, depois calcula a probabilidade da classe para cada registros das bases e por fim calcula a distância KS entre as distribuições das bases, retornando esse valor.

In [ ]:
def performance(regs, model):
    df = reading_json(regs)
    pretrained_model = pickle.load(open(model, 'rb'))
    meses = contador_meses(df)
    targets = df['TARGET'].astype(int)
    df = df.drop(columns=['TARGET', 'REF_DATE'])
    df_processed = pre_processing(df)
    performance_score = roc_auc_score(targets, pretrained_model.predict(df_processed))
    return performance_score, '; '.join(meses)

def aderencia(base, model):
    df_localbase = pd.read_csv(base, low_memory=False)
    df_teste = pd.read_csv("D:\\Neurotech\\datasets\\credit_01\\test", low_memory=False)
    pretrained_model = pickle.load(open(model, 'rb'))

    if 'train' in base:
        df_localbase = df_localbase.drop(columns=['ID', 'TARGET', 'REF_DATE'])
    else:
        df_localbase = df_localbase.drop(columns=['ID', 'REF_DATE'])
    
    df_teste = df_teste.drop(columns=['ID', 'TARGET', 'REF_DATE'])
    df_localbase_prep = pre_processing(df_localbase)
    df_teste_prep = pre_processing(df_teste)

    local_prob = probabilidades(df_localbase_prep, pretrained_model)
    teste_prob = probabilidades(df_teste_prep, pretrained_model)
    ks_distance = ks_2samp(local_prob, teste_prob)
    return getattr(ks_distance, 'statistic_location')